In [7]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [8]:
f = open(r"C:\Users\38061\Jupyter_Notebook\shujuwajue\TheTwo\数据集-T10I4D100K.txt")  
line = f.readline() 
lines = []
while line:  
    line_data = line.split()  
    # lines.append(np.array(line_data,dtype='int16'))
    lines.append(line_data)
    line = f.readline()  
f.close()
#data = np.array(lines)
data = lines
data

[['25',
  '52',
  '164',
  '240',
  '274',
  '328',
  '368',
  '448',
  '538',
  '561',
  '630',
  '687',
  '730',
  '775',
  '825',
  '834'],
 ['39', '120', '124', '205', '401', '581', '704', '814', '825', '834'],
 ['35', '249', '674', '712', '733', '759', '854', '950'],
 ['39', '422', '449', '704', '825', '857', '895', '937', '954', '964'],
 ['15',
  '229',
  '262',
  '283',
  '294',
  '352',
  '381',
  '708',
  '738',
  '766',
  '853',
  '883',
  '966',
  '978'],
 ['26', '104', '143', '320', '569', '620', '798'],
 ['7',
  '185',
  '214',
  '350',
  '529',
  '658',
  '682',
  '782',
  '809',
  '849',
  '883',
  '947',
  '970',
  '979'],
 ['227', '390'],
 ['71',
  '192',
  '208',
  '272',
  '279',
  '280',
  '300',
  '333',
  '496',
  '529',
  '530',
  '597',
  '618',
  '674',
  '675',
  '720',
  '855',
  '914',
  '932'],
 ['183',
  '193',
  '217',
  '256',
  '276',
  '277',
  '374',
  '474',
  '483',
  '496',
  '512',
  '529',
  '626',
  '653',
  '706',
  '878',
  '939'],
 ['161',
  

In [12]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        #节点名称
        self.name=nameValue
        #节点在该路径总共出现次数
        self.count=numOccur
        #nodeLink:下一个同名节点的地址,构成整个头表的链,方便对FP树的快速访问
        self.nodeLink=None
        #节点的前一个节点(父节点)
        self.parent=parentNode
        #节点的孩子节点,因为孩子节点的个数不确定,并非什么二叉树,所以用一个字典存放,key:子节点的名称,value:子节点的地址
        self.children={}

    def inc(self,numOccur):
        self.count+=numOccur

def createTree(dataSet,minSup):
    # 首先我们思考一下创建FP树的过程
    '''
    一般我们分两步：
    第一步:先扫描一遍数据集,过滤掉非频繁项，得到频繁1项集,同时需要存储它们出现的次数,因为第二遍扫描记录时，需要根据
    第二步:再一次扫描数据集，对每一条记录，去掉非频繁项，然后对记录中的项按出现次数从高到底排序，然后插入FP树
    '''
    # 第一步
    headerTable={}
    for T in dataSet:
        for item in T:
            headerTable[item]=headerTable.get(item,0)+dataSet[T]
    #去掉不满足最小支持度的item
    for k in headerTable.keys():
        if headerTable[k]<minSup:
            del headerTable[k]
    
    
    freqItemSet=set(headerTable.keys())
    if len(freqItemSet)==0:
        return None,None
    '''
    头指针表:FP-growth算法还需要头表，存储每个节点链的第一个节点地址，每个节点通过nodeLink指向下一个节点，
    这样就构成一个单链表
    '''
    # 初始化头表
    for k in headerTable:
        headerTable[k]=[headerTable[k],None]
    
    root=treeNode('Null Set',0,None)
    #第二次扫描数据集，对每一条记录先过滤掉非频繁项，后排序，再插入到FP树
    dataSet_tuple=dataSet.items()
    for tranSet,count in dataSet_tuple:
        localD={}
        for item in tranSet:
            if item in freqItemSet:
                localD[item]=headerTable[item][0]
        #item in localD are ordered
        #元素按照键值从大到小，关键字按照字典序排列
        if len(localD)>0:
            orderedItems = [v[0] for v in sorted(localD.items(),key = lambda p:(-p[1],p[0]))]
            # 插入到FP树中,每一条记录都是从根节点插入
            insert_treeNode(orderedItems,root,headerTable,count)
    return root,headerTable
 
def insert_treeNode(items, into_treeNode, headerTable, count):
    '''
    Create the FP-Tree
    Insert the node and update the HeaderTable
    '''
    if items[0] in into_treeNode.children:
        into_treeNode.children[items[0]].inc(count)
    else:
        into_treeNode.children[items[0]] = treeNode(items[0], count, into_treeNode) #初始化要插入节点,并插入到当前节点
        
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = into_treeNode.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], into_treeNode.children[items[0]]) #headerTable[items[0]][1]是一个树节点

    if len(items) > 1: 
        insert_treeNode(items[1:], into_treeNode.children[items[0]], headerTable, count) #此时插入的的位置从into_treeNode.children[items[0]]开始了

def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

def loadDat():
    '''
    Load the Data
    '''
    datalist = []
    file = open('test.txt')
    diction = {}
    
    while 1:
        line = file.readline()
        l = line[:-2].split(' ')
        l = l[2:]
        if l:
            datalist.append(l)
        for i in l:
            if i not in diction:
                diction[i] = 1
            else:
                diction[i] += 1
        if not line:
            break
        pass
    
    file.close() 
    return datalist
     
def createInitSet(dataSet):
    '''
    Create the Dict
    '''
    retDict={}
    for Tid in dataSet:
        retDict[frozenset(Tid)]=retDict.get(frozenset(Tid),0)+1
    return retDict
 
def findPrefixPath(basePat,treeNode):
    '''
    BasePat: the item
    TreeNode: the item in the HeadTable is the first node
    '''
    condPats = {} #condition pattern
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            #fronzenset as key!
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats
 
def ascendTree(treeNode, prefixPath): #找当前节点到根节点的路径(从下而上顺序),路径按item的名字存放在prefixPath
    if treeNode.parent != None:
        prefixPath.append(treeNode.name)
        ascendTree(treeNode.parent, prefixPath)

def CLOSET(FPtree, headerTable, minSup, preFix, closedDict, support):
    #minSup:支持度,closedDict:频繁闭项集,preFix:该项的前缀,FPtree:构建的FP树,headerTable:FP树对应的头表,
    #support : the preFix's support count
    #If the preFix's support is larger than all the condition pattern, it is the closed pattern
    #元素键值相同时，按照字典序排序
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p:(-p[1][0],p[0]),reverse = True)]  #从频次出现低项开始挖掘
    #如果前缀的支持度大于所有的superset的支持度，则前缀本身可能就是一个CLOSED ITEMSET
    if support > headerTable[bigL[-1]][0]:
        flag = True
        for k in closedDict:
            if frozenset(preFix).issubset(k) and closedDict[k] >= support:
                flag = False
        if flag == True:
            closedDict[frozenset(preFix)]=support

    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])    #求条件模式基
        myCondTree,myHead = createTree(condPattBases, minSup)
        if myHead == None:
            flag = True
            for k in closedDict:

                if frozenset(newFreqSet).issubset(k) and closedDict[k] >= headerTable[basePat][0]:
                    flag = False
            if flag == True:
                closedDict[frozenset(newFreqSet)]=headerTable[basePat][0]
    #Mine the closed frequent itemsets using CLOSET recursively
        if myHead != None:
            CLOSET(myCondTree, myHead, minSup, newFreqSet, closedDict, headerTable[basePat][0]) #FP树中的递归有一个特点:没有返回值,需要记录的数据都放在参数中了,上层可以直接拿到数据

dataSet = data
initSet  = createInitSet(dataSet)
minsup = 0.4 * len(dataSet)
FPtree,HeaderTab = createTree(initSet,minsup)
freqDict={}
CLOSET(FPtree,HeaderTab,minsup,set([]),freqDict, minsup)
print(freqDict)

RuntimeError: dictionary changed size during iteration

In [13]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpmax, fpgrowth
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [16]:
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)

fi2 = fpgrowth(df, min_support=0.001, use_colnames=True,verbose=1)
#frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.01, use_colnames=True)
print('以下为support>0.001的情况下频繁项集')
fi2

797 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (368)
25 itemset(s) from tree conditioned on items (538)
0 itemset(s) from tree conditioned on items (538, 368)
0 itemset(s) from tree conditioned on items (538, 529)
0 itemset(s) from tree conditioned on items (538, 120)
0 itemset(s) from tree conditioned on items (538, 829)
0 itemset(s) from tree conditioned on items (538, 789)
0 itemset(s) from tree conditioned on items (538, 494)
1 itemset(s) from tree conditioned on items (538, 460)
0 itemset(s) from tree conditioned on items (538, 722)
0 itemset(s) from tree conditioned on items (538, 766)
0 itemset(s) from tree conditioned on items (538, 438)
0 itemset(s) from tree conditioned on items (538, 937)
0 itemset(s) from tree conditioned on items (538, 217)
0 itemset(s) from tree conditioned on items (538, 914)
0 itemset(s) from tree conditioned on items (538, 684)
0 itemset(s) from tree conditioned on items (538, 692)
0 itemset(s) from tree co

11 itemset(s) from tree conditioned on items (240)
0 itemset(s) from tree conditioned on items (240, 538)
1 itemset(s) from tree conditioned on items (240, 274)
0 itemset(s) from tree conditioned on items (240, 470)
1 itemset(s) from tree conditioned on items (240, 597)
2 itemset(s) from tree conditioned on items (240, 75)
0 itemset(s) from tree conditioned on items (240, 75, 470)
1 itemset(s) from tree conditioned on items (240, 75, 597)
0 itemset(s) from tree conditioned on items (240, 956)
1 itemset(s) from tree conditioned on items (240, 541)
2 itemset(s) from tree conditioned on items (240, 236)
0 itemset(s) from tree conditioned on items (240, 236, 541)
1 itemset(s) from tree conditioned on items (240, 236, 956)
3 itemset(s) from tree conditioned on items (240, 17)
0 itemset(s) from tree conditioned on items (240, 17, 541)
1 itemset(s) from tree conditioned on items (240, 17, 956)
2 itemset(s) from tree conditioned on items (240, 17, 236)
0 itemset(s) from tree conditioned on ite

6 itemset(s) from tree conditioned on items (704)
0 itemset(s) from tree conditioned on items (704, 39)
1 itemset(s) from tree conditioned on items (704, 825)
0 itemset(s) from tree conditioned on items (704, 529)
0 itemset(s) from tree conditioned on items (704, 829)
0 itemset(s) from tree conditioned on items (704, 722)
0 itemset(s) from tree conditioned on items (704, 368)
15 itemset(s) from tree conditioned on items (814)
0 itemset(s) from tree conditioned on items (814, 844)
0 itemset(s) from tree conditioned on items (814, 966)
1 itemset(s) from tree conditioned on items (814, 196)
2 itemset(s) from tree conditioned on items (814, 674)
0 itemset(s) from tree conditioned on items (814, 674, 196)
1 itemset(s) from tree conditioned on items (814, 674, 966)
0 itemset(s) from tree conditioned on items (814, 788)
0 itemset(s) from tree conditioned on items (814, 614)
1 itemset(s) from tree conditioned on items (814, 57)
2 itemset(s) from tree conditioned on items (814, 658)
0 itemset(s

15 itemset(s) from tree conditioned on items (712)
0 itemset(s) from tree conditioned on items (712, 854)
1 itemset(s) from tree conditioned on items (712, 674)
0 itemset(s) from tree conditioned on items (712, 829)
1 itemset(s) from tree conditioned on items (712, 296)
2 itemset(s) from tree conditioned on items (712, 234)
0 itemset(s) from tree conditioned on items (712, 234, 296)
1 itemset(s) from tree conditioned on items (712, 234, 829)
3 itemset(s) from tree conditioned on items (712, 649)
0 itemset(s) from tree conditioned on items (712, 649, 234)
1 itemset(s) from tree conditioned on items (712, 649, 296)
2 itemset(s) from tree conditioned on items (712, 649, 829)
0 itemset(s) from tree conditioned on items (712, 649, 829, 296)
1 itemset(s) from tree conditioned on items (712, 649, 829, 234)
4 itemset(s) from tree conditioned on items (712, 355)
0 itemset(s) from tree conditioned on items (712, 355, 649)
1 itemset(s) from tree conditioned on items (712, 355, 296)
2 itemset(s) f

0 itemset(s) from tree conditioned on items (966, 914)
0 itemset(s) from tree conditioned on items (966, 438)
0 itemset(s) from tree conditioned on items (966, 32)
1 itemset(s) from tree conditioned on items (966, 368)
0 itemset(s) from tree conditioned on items (966, 529)
0 itemset(s) from tree conditioned on items (966, 692)
0 itemset(s) from tree conditioned on items (966, 39)
0 itemset(s) from tree conditioned on items (966, 494)
0 itemset(s) from tree conditioned on items (966, 829)
0 itemset(s) from tree conditioned on items (966, 362)
0 itemset(s) from tree conditioned on items (966, 937)
0 itemset(s) from tree conditioned on items (966, 145)
0 itemset(s) from tree conditioned on items (966, 789)
26 itemset(s) from tree conditioned on items (381)
0 itemset(s) from tree conditioned on items (381, 766)
0 itemset(s) from tree conditioned on items (381, 966)
0 itemset(s) from tree conditioned on items (381, 283)
0 itemset(s) from tree conditioned on items (381, 722)
0 itemset(s) fro

...

0 itemset(s) from tree conditioned on items (896, 461, 529, 829, 75, 403)
1 itemset(s) from tree conditioned on items (896, 461, 529, 829, 75, 950)
3 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205)
0 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205, 950)
1 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205, 403)
2 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205, 75)
0 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205, 75, 403)
1 itemset(s) from tree conditioned on items (896, 461, 529, 829, 205, 75, 950)
5 itemset(s) from tree conditioned on items (896, 461, 529, 285)
0 itemset(s) from tree conditioned on items (896, 461, 529, 285, 403)
1 itemset(s) from tree conditioned on items (896, 461, 529, 285, 75)
2 itemset(s) from tree conditioned on items (896, 461, 529, 285, 950)
0 itemset(s) from tree conditioned on items (896, 461, 529, 285, 950, 403)
1 itemset(s) from tree conditioned on items (896, 4

0 itemset(s) from tree conditioned on items (102)
3 itemset(s) from tree conditioned on items (714)
0 itemset(s) from tree conditioned on items (714, 229)
1 itemset(s) from tree conditioned on items (714, 692)
2 itemset(s) from tree conditioned on items (714, 422)
0 itemset(s) from tree conditioned on items (714, 422, 229)
1 itemset(s) from tree conditioned on items (714, 422, 692)
3 itemset(s) from tree conditioned on items (828)
0 itemset(s) from tree conditioned on items (828, 177)
1 itemset(s) from tree conditioned on items (828, 90)
0 itemset(s) from tree conditioned on items (828, 120)
11 itemset(s) from tree conditioned on items (218)
0 itemset(s) from tree conditioned on items (218, 368)
0 itemset(s) from tree conditioned on items (218, 640)
0 itemset(s) from tree conditioned on items (218, 701)
0 itemset(s) from tree conditioned on items (218, 540)
0 itemset(s) from tree conditioned on items (218, 201)
0 itemset(s) from tree conditioned on items (218, 675)
1 itemset(s) from tr

6 itemset(s) from tree conditioned on items (898)
0 itemset(s) from tree conditioned on items (898, 190)
1 itemset(s) from tree conditioned on items (898, 581)
2 itemset(s) from tree conditioned on items (898, 163)
0 itemset(s) from tree conditioned on items (898, 163, 581)
1 itemset(s) from tree conditioned on items (898, 163, 190)
3 itemset(s) from tree conditioned on items (898, 782)
0 itemset(s) from tree conditioned on items (898, 782, 163)
1 itemset(s) from tree conditioned on items (898, 782, 581)
2 itemset(s) from tree conditioned on items (898, 782, 190)
0 itemset(s) from tree conditioned on items (898, 782, 190, 163)
1 itemset(s) from tree conditioned on items (898, 782, 190, 581)
4 itemset(s) from tree conditioned on items (898, 25)
0 itemset(s) from tree conditioned on items (898, 25, 163)
1 itemset(s) from tree conditioned on items (898, 25, 581)
2 itemset(s) from tree conditioned on items (898, 25, 190)
0 itemset(s) from tree conditioned on items (898, 25, 190, 581)
1 ite

以下为support>0.001的情况下频繁项集


,support,itemsets
0,0.07828,(368)
1,0.03982,(538)
2,0.03771,(775)
3,0.03085,(825)
4,0.02783,(561)
...,...,...
27527,0.00182,"(640, 259, 886, 890, 38, 806)"
27528,0.00181,"(722, 259, 886, 890, 38, 806)"
27529,0.00180,"(640, 722, 259, 886, 38, 806)"
27530,0.00179,"(640, 722, 259, 886, 890, 38, 806)"


In [19]:
'''
def creatC1(dataset):
    C1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))
#frozenset 是不可变类型，可以将集合作为字典健值使用，set不能；

def scanD(Dset, Ck, minSupport):
#Dset是数据集，Ck是候选项集列表，minSupport是最小支持度
    ssCnt = {}
    #创建一个空字典ssCnt用来保存候选子集和候选子集对应的频数
    for tid in Dset:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.__contains__(can):
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
                    
   #计算字典中的每个元素，计算支持度                 
    numItems = float(len(Dset))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.inset(0, key)
        supportData[key] = support
        
    return retList, supportData
#retList是返回的频繁项集， supportData是对应的的支持度

def aprioriGen(Lk, k):
#Lk是频繁项集列表，K是项集元素个数，函数返回的是CK，即含有K个元素的候选项集
    retlist = []
    lenLk = len(Lk)
#计算频繁项集的元素个数，然后比较项集中每一个元素和其他元素（两个for循环）
#如果一个元素和另一个元素的前k-2项重复，则将两个元素合并成一个大小为K的集合
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retlist.append(Lk[i] | Lk[j]) #（并集操作）
    return retlist

def apriori(dataset, minsupport = 0.5):
    C1 = creatC1(dataset)
    D = list(map(set, dataset))
    L1, supportdata = scanD(D,C1 ,minsupport)
    L = [L1]
#先调用createC1和scanD产生只有一个元素的频繁项集
 
#然后基于L调用apriorigen产生只有两个元素的候选集，再调用scanD来生成含有
#两个元素的频繁项集，之后K加一，循环调用产生含有三个元素的频繁项集，一直循环
#直到L(K-2)的长度=0
    k =2
    while (len(L[k-2]) > 0):
        Ck = apriorigen(L[k-2], k)
        Lk, supk = scanD(D, Ck, 0.2)
        supportdata.update(supk)
        L.append(Lk)
        k += 1
    return L, supportdata

L, SupportData = apriori(data,minsupport=0.7)
SupportData
'''

In [10]:
	import sys
	import time
	'''''
	def loadData():
	    itemset=[]
	    cf=open("D:\\data\\traindata02.data")
	    lines =cf.readlines()
	    for line in lines:
	        itemset.append(line)
	    return itemset
	'''''
	def loadData():
	    return [[1,3,4,5,6],[1,2,5],[3,5,6],[1,3,4,6],[3,5,6]]
	def find_frequent_1_itemsets(D, minsupport):
	    L1 = []
	    C1 = []
	    cnt = {}
	    for transcation in D:
	        for item in transcation:
	            if not (item=='\n' or item==' '):
	                if not [item] in C1:
	                    C1.append([item])
	                    cnt[item] = 1
	                else:
	                    cnt[item] += 1
	    for transcation in C1:
	        for item in transcation:
	            if cnt[item] >= minsupport:
	                L1.append(transcation)               
	    L1.sort()
	    return L1  #return all itemset in the same level
	
	def aproiri_gen(L, L1, D , minsupport): #生成新一层的所有节点
	    res = []
	    for i in range(len(L)):
	        for j in range(len(L1)):
	            if L[i][-1] < L1[j][0]:   #L[i][-1]是L列表中，第i项最后一个字母
	                candidate = list(set(L[i]).union(set(L1[j]))) #union             
	                cnt = 0
	                for transcation in D: 
	                    if compareList(candidate, transcation):
	                        cnt += 1 
	                if cnt >= minsupport:
	                    res.append(candidate)
	
	    return res
	
	def compareList(l1, l2): #l1的每一项都在l2中
	    for item in l1:
	        if item not in l2:
	            return False
	    return True
	                
	def Aproiri(D, minsupport):
	    L = []
	    close=[]
	    L1 = find_frequent_1_itemsets(D, minsupport) #第一层的候选项集
	    #print('第 1 层的候选项集为：',L1)
	    L.append([])
	    L.append(L1)
	    
	    for k in range(2,len(L1)):
	        Lk = []
	        if len(L[k-1]) == 0:
	            break
	        Ck = aproiri_gen(L[k-1] ,L1,D,minsupport)
	        #print("第",k,"层的候选项集为:" , Ck)
	        L.append(Ck)
	    
	    for i in range(len(L)):
	        for item in L[i]:
	            close.append(item)
	    #print("频繁项集个数为：",len(close)-1)
	
	      #  maximal.append(maximalE)
	        #close.append(closeE)
	    for k in range(2,len(L1)):
	        if len(L[k-1]) == 0:
	            break
	        for transcation in L[k-1]:
	            cnt = 0
	            for item in D:
	                if compareList(transcation, item):
	                    cnt +=1
	            for candidate in L[k]:
	                if compareList(transcation, candidate):
	                    cnm = 0
	                    for item in D:
	                        if compareList(candidate, item):
	                            cnm +=1
	                    if cnt <= cnm:
	                        if transcation in close:
	                            close.remove(transcation)
	                            
	    return L,close
	#test
	start=time.time()
	D = data
	
	L ,close = Aproiri(D, 2000)
	print ("频繁项集为：",L)
	#print ("闭合项集个数为：",len(close))
	print("闭合项集为：",close)
	end=time.time()
	print ('程序运行时间为：',end-start,'s')

KeyboardInterrupt: 

In [19]:
from apriori_python import apriori

freqItemSet, rules = apriori(data, minSup=0.01, minConf=0.5)
print(rules,freqItemSet) 

KeyboardInterrupt: 

In [ ]:
import apriori

for rule in apriori.generate_rules(data, min_support=0.06):
    msg = (f'{rule.format_rule():20s}\t\t'
           f'(support={rule.support:0.4f}, confidence={rule.confidence:0.4f}, lift={rule.lift:0.4f})')
    print(msg)